In [9]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [10]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
0,0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197,5호선 상일동,373.362877,NaN
1,12,28480888,옥돌영양탕,NaN,Q,음식,Q05,닭/오리요리,Q05A01,삼계탕전문,...,143220,4945.0,NaN,1,NaN,127.089673,37.557977,5호선 아차산(어린이대공원후문),354.921957,NaN
2,14,17175285,은진이네,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,121861,4206.0,NaN,NaN,NaN,126.956164,37.551340,5호선 애오개,53.083100,NaN
3,15,17175333,흥부보쌈,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,158721,7997.0,NaN,5,NaN,126.875987,37.529132,5호선 오목교(목동운동장앞),353.540138,NaN
4,28,20665661,가파도참치,NaN,Q,음식,Q03,일식/수산물,Q03A04,참치전문점,...,158050,8005.0,NaN,2,NaN,126.873242,37.524340,5호선 오목교(목동운동장앞),122.836352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,135800,17212093,만랩커피,오금역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138800,5702.0,NaN,NaN,103,127.125353,37.502381,5호선 오금,124.273859,NaN
20045,135805,17218896,얼음과자,NaN,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,157857,7510.0,NaN,1,123,126.812418,37.577436,5호선 방화,27.430119,NaN
20046,135806,17218871,오이소서울,상일점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134090,5276.0,NaN,1,104,127.165574,37.550924,5호선 상일동,983.009054,NaN
20047,135807,17218873,얼음왕국명일14,6호점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134780,5297.0,상가,1,106,127.146217,37.546317,5호선 굽은다리(강동구민회관앞),347.273163,NaN


In [11]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,NaN,상일동,상일동역 도전최강달인왕만두
1,옥돌영양탕,닭/오리요리,삼계탕전문,서울특별시 광진구 긴고랑로36길 24,5호선 아차산(어린이대공원후문),354.921957,NaN,아차산,아차산역 옥돌영양탕
2,은진이네,한식,한식/백반/한정식,서울특별시 마포구 마포대로 196,5호선 애오개,53.083100,NaN,애오개,애오개역 은진이네
3,흥부보쌈,한식,족발/보쌈전문,서울특별시 양천구 목동동로 309,5호선 오목교(목동운동장앞),353.540138,NaN,오목교,오목교역 흥부보쌈
4,가파도참치,일식/수산물,참치전문점,서울특별시 양천구 오목로54길 2,5호선 오목교(목동운동장앞),122.836352,NaN,오목교,오목교역 가파도참치
...,...,...,...,...,...,...,...,...,...
20044,만랩커피,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 중대로23길 21,5호선 오금,124.273859,NaN,오금,오금역 만랩커피
20045,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,NaN,방화,방화역 얼음과자
20046,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,NaN,상일동,상일동역 오이소서울
20047,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,NaN,굽은다리,굽은다리역 얼음왕국명일14


In [12]:
items = data["검색어"][8151:]
items

8151             화곡역 맛집
8152          상일동역 수라면가
8153        영등포시장역 고창김밥
8154        아차산역 스마트글로벌
8155           신정역 카페다온
              ...      
20044          오금역 만랩커피
20045          방화역 얼음과자
20046        상일동역 오이소서울
20047    굽은다리역 얼음왕국명일14
20048         상일동역 얼음과자
Name: 검색어, Length: 11898, dtype: object

In [13]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}


In [14]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path=r'C:\Users\정예림\Desktop\텍스트마이닝\chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('link_photo > img') #이미지
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")


진행상황 : 1/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/11898
식당 존재 x
진행상황 : 5/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/11898
식당 존재 x
진행상황 : 7/11898
식당 존재 x
진행상황 : 8/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/11898
식당 존재 x
진행상황 : 17/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 18/11898
식당 존재 x
진행상황 : 19/11898
식당 존재 x
진행상황 : 20/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 21/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 22/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 23/11898
식당 존재 x
진행상황 

식당 존재 x
진행상황 : 188/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 189/11898
식당 존재 x
진행상황 : 190/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 191/11898
식당 존재 x
진행상황 : 192/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 193/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 194/11898
식당 존재 x
진행상황 : 195/11898
식당 존재 x
진행상황 : 196/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 197/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 198/11898
식당 존재 x
진행상황 : 199/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 200/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 201/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 202/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 203/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 204/11898
식당 존재 x
진행상황 : 205/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 206/11898
식당 존재 x
진행상황 : 207/11898
식당 존재 x
진행상황 : 208/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 209/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 210/11898
식당 존재 x

식당 존재 x
진행상황 : 370/11898
식당 존재 x
진행상황 : 371/11898
식당 존재 x
진행상황 : 372/11898
식당 존재 x
진행상황 : 373/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 374/11898
식당 존재 x
진행상황 : 375/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 376/11898
식당 존재 x
진행상황 : 377/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 378/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 379/11898
식당 존재 x
진행상황 : 380/11898
식당 존재 x
진행상황 : 381/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 382/11898
식당 존재 x
진행상황 : 383/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 384/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 385/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 386/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 387/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 388/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 389/11898
식당 존재 x
진행상황 : 390/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 391/11898
식당 존재 x
진행상황 : 392/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 393/11898
식당 존재
0


식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 549/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 550/11898
식당 존재 x
진행상황 : 551/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 552/11898
식당 존재 x
진행상황 : 553/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 554/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 555/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 556/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 557/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 558/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 559/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 560/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 561/11898
식당 존재 x
진행상황 : 562/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 563/11898
식당 존재 x
진행상황 : 564/11898
식당 존재 x
진행상황 : 565/11898
식당 존재 x
진행상황 : 566/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 567/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 568/11898
식당 존재 x
진행상황 : 569/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 570/118

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 725/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 726/11898
식당 존재 x
진행상황 : 727/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 728/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 729/11898
식당 존재 x
진행상황 : 730/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 731/11898
식당 존재 x
진행상황 : 732/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 733/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 734/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 735/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 736/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 737/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 738/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 739/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 740/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 741/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 742/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 743/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 744/11898
식당 존

식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 902/11898
식당 존재 x
진행상황 : 903/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 904/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 905/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 906/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 907/11898
식당 존재 x
진행상황 : 908/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 909/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 910/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 911/11898
식당 존재 x
진행상황 : 912/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 913/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 914/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 915/11898
식당 존재 x
진행상황 : 916/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 917/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 918/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 919/11898
식당 존재 x
진행상황 : 920/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 921/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 922/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가

식당 존재 x
진행상황 : 1082/11898
식당 존재 x
진행상황 : 1083/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1084/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1085/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1086/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1087/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1088/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1089/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1090/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1091/11898
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1092/11898
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1093/11898
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1094/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1095/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1096/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1097/11898
식당 존재 x
진행상황 : 1098/11898
식당 존재 x
진행상황 : 1099/11898
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1100/11898
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1101/11898
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1102/

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1262/11898
식당 존재
4


KeyboardInterrupt: 

In [15]:
img_elem

,ItemID,img_url
0,화곡역 맛집,None
1,화곡역 맛집,None
2,화곡역 맛집,None
3,화곡역 맛집,None
4,화곡역 맛집,None
...,...,...
4736,청구역 은화계,None
4737,답십리역 유노초밥,None
4738,답십리역 유노초밥,None
4739,답십리역 유노초밥,None


In [16]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,10i1tsl,화곡역 맛집,5점,2021.03.14.
1,11b08qg,화곡역 맛집,5점,2020.12.23.
2,dvjq73,화곡역 맛집,2점,2020.12.15.
3,v32d49,화곡역 맛집,5점,2020.07.21.
4,13e9ti9,화곡역 맛집,2점,2020.07.19.
...,...,...,...,...
4736,96f6iu,청구역 은화계,5점,2020.02.01.
4737,v7o8el,답십리역 유노초밥,4점,2020.12.29.
4738,19pe0n0,답십리역 유노초밥,5점,2020.11.22.
4739,12tf9bb,답십리역 유노초밥,5점,2020.10.05.


In [17]:
img_elem

,ItemID,img_url
0,화곡역 맛집,None
1,화곡역 맛집,None
2,화곡역 맛집,None
3,화곡역 맛집,None
4,화곡역 맛집,None
...,...,...
4736,청구역 은화계,None
4737,답십리역 유노초밥,None
4738,답십리역 유노초밥,None
4739,답십리역 유노초밥,None


In [18]:
review_elem

,ItemID,review
0,화곡역 맛집,쌈도 다양하고 신선하고 우렁도 쫄깃했어요~ 제가 갈치속젓을 정말이지 좋아해서 너무 ...
1,화곡역 맛집,야채도 신선하고 생오리쌈밥 정말 최고네요~ 맛있게 잘먹고갑니다!
2,화곡역 맛집,제육은 돼지냄새가 좀..
3,화곡역 맛집,상차림이 정갈하고 맛깔납니다. 다양한 야채를 먹을 수 있어 건강한 식사를 할 수 있...
4,화곡역 맛집,제육은 맛이 그냥저냥이네요
...,...,...
4736,청구역 은화계,목살..최고임
4737,답십리역 유노초밥,이 동네에서 제일!!
4738,답십리역 유노초밥,맛있고 깔끔하고 친절하고 저렴한 편이고 게다가 날치알 롤 서비스 주심 조오오오온맛
4739,답십리역 유노초밥,와 여기는 성공할 수밖에 없는 맛집이에요 가게 내부도 깔끔하고 초밥 하나 하나가 다...


In [19]:
img_elem.head()

,ItemID,img_url
0,화곡역 맛집,None
1,화곡역 맛집,None
2,화곡역 맛집,None
3,화곡역 맛집,None
4,화곡역 맛집,None


In [20]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID     ItemID Rating    Timestamp
4737   v7o8el  답십리역 유노초밥     4점  2020.12.29.
4738  19pe0n0  답십리역 유노초밥     5점  2020.11.22.
4739  12tf9bb  답십리역 유노초밥     5점  2020.10.05.
4740     None  답십리역 유노초밥   None  2020.10.05.
         ItemID img_url
4737  답십리역 유노초밥    None
4738  답십리역 유노초밥    None
4739  답십리역 유노초밥    None
4740  답십리역 유노초밥    None
         ItemID                                             review
4737  답십리역 유노초밥                                        이 동네에서 제일!!
4738  답십리역 유노초밥      맛있고 깔끔하고 친절하고 저렴한 편이고 게다가 날치알 롤 서비스 주심 조오오오온맛
4739  답십리역 유노초밥  와 여기는 성공할 수밖에 없는 맛집이에요 가게 내부도 깔끔하고 초밥 하나 하나가 다...
4740  답십리역 유노초밥  와 여기는 성공할 수밖에 없는 맛집이에요 가게 내부도 깔끔하고 초밥 하나 하나가 다...


In [22]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_rating(10318).csv')

In [23]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_User_ID(10318).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_img(10318).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_review(10318).csv')

In [19]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


1101

In [100]:
print (data.loc[data['검색어'] == "서대문역 아웃백스테이크하우스"])

             상호명 상권업종중분류명 상권업종소분류명             도로명주소       역명          거리  \
2527  아웃백스테이크하우스       양식  패밀리레스토랑  서울특별시 중구 통일로 114  5호선 서대문  122.856647   

      카카오url    역              검색어  
2527     NaN  서대문  서대문역 아웃백스테이크하우스  


IndexError: list index out of range